In [2]:
# Sam Brown
# sam_brown@mines.edu
# July 14
# Goal: Investigate different ways to calculate the energy of a slip event and look for trends mg stations

import pandas as pd
import numpy as np
from Stations import Station
import my_lib.funcs
import matplotlib.pyplot as plt

from netCDF4 import Dataset
import xarray as xr

In [4]:
# Load in antarctica ice depth data
filepath = "/Users/sambrown04/Documents/SURF/NSIDC-0756_3-20250708_204745/BedMachineAntarctica-v3.nc"
ds = xr.open_dataset(filepath)

In [7]:
mg01x = -178462.5466582153 
mg01y = -555835.6719349005

mg06x = -189291.11945724243 
mg06y = -570212.4036758058

mg07x = -185263.18872241525 
mg07y = -558217.6317621776

mg04x = -181853.9877418033 
mg04y = -560788.2678059591


In [13]:
coords = {
    'mg01': (-178462.5466582153, -555835.6719349005),
    'mg04': (-181853.9877418033,  -560788.2678059591),
    'mg06': (-189291.11945724243, -570212.4036758058),
    'mg07': (-185263.18872241525, -558217.6317621776)
}

for label, (x_val, y_val) in coords.items():
    thickness = ds['thickness'].sel(x=x_val, y=y_val, method='nearest')
    print(f"{label}: {thickness.values:.2f} meters")

mg01: 872.15 meters
mg04: 831.70 meters
mg06: 758.22 meters
mg07: 855.19 meters


Shoelace formula:
$$
A = \frac{1}{2} \left| \sum_{i=1}^{n} (x_i y_{i+1} - x_{i+1} y_i) \right|
$$

In [19]:
avg = 829.32

# Use a triangle to calculate the area and the volume using shoelace formula
x = np.array([
    -178462.5466582153,  # mg01
    -189291.11945724243, # mg06
    -185263.18872241525, # mg07
    -181853.9877418033   # mg04
])

y = np.array([
    -555835.6719349005,  # mg01
    -570212.4036758058,  # mg06
    -558217.6317621776,  # mg07
    -560788.2678059591   # mg04
])

# append the first point to the end
x_closed = np.append(x, x[0])
y_closed = np.append(y, y[0])


sum1 = np.sum(x_closed[:-1] * y_closed[1:])
sum2 = np.sum(y_closed[:-1] * x_closed[1:])
area_m2 = 0.5 * np.abs(sum1 - sum2)


print(f"Area: {area_m2:.2f} square meters")

Area: 23187612.92 square meters


In [30]:
vol = area_m2 * avg
density = 910 # kg/m^3
mass = vol * density

In [36]:
# We will use 2014 data because that is the only time these stations were up consistently 
df14 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2014_2014Events2stas")
df14 = my_lib.funcs.preprocess_events(df14)

In [40]:
# Filter so it is only mg stations
mg_dat = []

for i, event in enumerate(df14):

    filt_df = pd.DataFrame()
    
    for col in event.columns:
        colname = col[0:2]
        if colname == 'mg' or colname == 'ti' or colname == 'ti':
            column = event[col].copy()
            filt_df[col] = column
            
    mg_dat.append(filt_df)

In [48]:
f_events = []
derivs_events = []
energies = pd.DataFrame(columns=['date', 'evt_energy'])

dt = 15  # Time step in seconds
mass = mass  # Assumed to be defined earlier

for i, event in enumerate(mg_dat):
    cols = [col for col in event.columns if col not in ['time_sec', 'time_dt']]
    event_onsets = []

    for col in cols:
        # Compute first and second derivatives
        grad = my_lib.funcs.derivative(event[col].values)
        grad2 = my_lib.funcs.derivative(grad)

        # Identify max of second derivative (absolute value)
        max_idx = np.argmax(np.abs(grad2))
        max_time = event['time_sec'].iloc[max_idx]
        event_onsets.append(max_time)

    # Skip if no valid onset found
    if not event_onsets:
        print(f"Skipping event {i}: no valid onsets found.")
        continue

    # Store date from original (unspliced) event
    date = event['time_dt'].iloc[0]

    # Determine the release time as the max of onsets
    release_time = max(event_onsets)

    # Splice the event starting from release_time
    spliced_event = event[event['time_sec'] >= release_time].reset_index(drop=True)

    # Skip if spliced event too short
    if spliced_event.shape[0] < 50:
        print(f"Skipping short event {i}: only {spliced_event.shape[0]} samples.")
        continue

    # Compute average displacement
    cols_avg = [col for col in spliced_event.columns if col not in ['time_sec', 'time_dt']]
    spliced_event['avg_disp'] = spliced_event[cols_avg].mean(axis=1)

    # Compute derivative of average displacement
    deriv = my_lib.funcs.derivative(spliced_event['avg_disp'])

    f_events.append(spliced_event)

    # Store (date, derivative) pair
    derivs_events.append((date, deriv))

    # Compute and store energy
    evt_E = event_energy(deriv, mass, dt)
    energies.loc[len(energies)] = {
        "date": date,
        "evt_energy": evt_E
    }

Skipping event 26: no valid onsets found.
Skipping event 28: no valid onsets found.
Skipping short event 33: only 2 samples.
Skipping event 37: no valid onsets found.
Skipping event 42: no valid onsets found.
Skipping event 48: no valid onsets found.
Skipping event 59: no valid onsets found.
Skipping event 61: no valid onsets found.
Skipping event 82: no valid onsets found.
Skipping short event 91: only 8 samples.
Skipping event 98: no valid onsets found.
Skipping event 119: no valid onsets found.
Skipping event 131: no valid onsets found.
Skipping event 133: no valid onsets found.
Skipping event 135: no valid onsets found.
Skipping event 140: no valid onsets found.
Skipping event 155: no valid onsets found.
Skipping short event 170: only 1 samples.
Skipping event 180: no valid onsets found.
Skipping short event 189: only 3 samples.
Skipping event 210: no valid onsets found.
Skipping short event 215: only 8 samples.
Skipping short event 219: only 16 samples.
Skipping event 224: no vali

In [46]:
def event_energy(velocity_arr, mass, dt=15):
    kinetic_energy = 0.5 * mass * velocity_arr**2  # Joules at each step
    total_energy = np.sum(kinetic_energy) * dt       # Approximate integration
    return total_energy  # in joules

In [59]:
energies.shape

(466, 2)

In [61]:
energies.to_csv('Energies_mg', index = False)